In [1]:
from rap_db import *
import re
from os import listdir
from os.path import isfile, join
import nltk
#nltk.download()
estconn = pg2.connect(database='rap_songs', user='keenan', host='localhost', password='keenan')

In [2]:
#need this to figure out verses
cur = estconn.cursor()
cur.execute('''SELECT artist_nm FROM all_artist_names;''')
quer = cur.fetchall()
COMPLETE_RAPPERS = set(art[0].lower() for art in quer)

cur.execute('''SELECT artist_name FROM artists;''')
quer = cur.fetchall()
SCRAPED_RAPPERS = set(art[0] for art in quer)
cur.close()

create_music_tables(estconn, bypass = True)
bulk_load(estconn, ['mf_doom', 'chance_the_rapper'])

In [3]:
#need to find the best way to mark of important parts of the song,
# {**} == not sure but one thing is can mean someone saying something
# {?} == dont know line
# [Artist] == i think this is when an artist s speaking
# [yes] == can also mean like () whe someone says somthing, have to use [Capital First letter] to differentiate
#or a great way to differentiate is to check contents of [] against db of artists with tolower and edge case for chorus or outro or intro
# * == annotator notes
# (action!) == i think this is sound of action, can also be someone saying something in track
# () == someone saying something or repeat signal
# (**) ==
# looks like () and [] and {} are pretty much interchanchable and it matters a lot more whats inside
# [???] == i think it means they didn't know
# "" == i think this is when someone else is talking over the track
# Man1: denoting that someone is speaking in a conversation
#words = re.sub("([\(\[]).*?([\)\]])", "", lyrics)
#words = re.sub("\n","--",words)


In [4]:
class text_segment():
    def __init__(self, typ, label, start, end):
        self.typ = typ
        self.label = label
        self.start = start
        self.end = end
    def gen_content(self, raw_text, next_start):
        self.content = raw_text[self.end:next_start]

In [11]:
class song():
    #can consolidate this later if there's no benefit to differentiating, but cant have start with [ and end with ] or }
    #regex are dope
    regex_commands = {'intro':'\[.*intro.*\]',
                      'outro':'\[.*outro.*\]',
                      #saw (vhorus) and [vhorus] - just being careful
                      'chorus':'(\[.*chorus.*\]|\(.*chorus.*\))',
                      'verse':'(\[.*verse.*|.*bridge.*\]|\(.*verse.*|.*bridge.*\))',
                      '[]':'\[(?!.*intro.*|.*outro.*|.*chorus.*|.*verse.*|.*bridge.*).*\]',
                      '{}':'\{(.*?)\}',
                      '\n':'\n',
                      #still pulling others, may come back to this later
                      #'**':'\*(.*?)\*',
                      '{**}':'\{\*(.*?)\*\}',
                      '()':'\((?!.*chorus.*|.*verse.*|.*bridge.*).*\)',
                      '""':'"(.*?)"',
                      '?':'\?+',
                      '*text':'[\n| ]\*[^\n|\{|\(|\[]*'}
                        #second part stops on \n,{,[, or (, might need to add/take away more exceptions
    
    def __init__(self, raw_text, name):
        self.raw_text = raw_text
        self.name = name
    
    #basically want to go through dic and use a re ex to pull out the matching phrase and mark the spot in the string
    #with the marks, I can cut up the lyrics accordingly, I use extras_set/extras_list to rank my segments in order
    def assign_extras(self):
        #record all special instances
        self.extras = dict(zip(song.regex_commands.keys(), [[]]*len(song.regex_commands)))
        #this will later be used to create song segments
        extra_segs = set()
        
        for ext, regex_command in song.regex_commands.items():
            finder = re.compile(regex_command)
            ap_list = []
            for match in finder.finditer(self.raw_text.lower()):
                #match has a start pos and a match value, add to list to append
                seg_type = ext
                if match.group()[1:-1] in COMPLETE_RAPPERS:
                    #this if statement is a catch all for rappers names mentioned to be sent to verse
                    #wanna check things like '[eminem]' so [1:-1]
                    seg_type = 'verse'
                    self.extras[seg_type].append((match.group(), (match.start(), match.start()+len(match.group()))))
                else:
                    ap_list.append((match.group(), (match.start(), match.start()+len(match.group()))))
                
                #determines what segments will be used to construct actual song
                #basically each segment has start and end and song willbe build on raw text before and after these anchor points
                if seg_type in ['verse', 'chorus', 'intro', 'outro', '[]', '()']:
                    extra_segs = extra_segs|{((seg_type, match.group()),(match.start(), match.start()+len(match.group())))}
            self.extras[ext] = ap_list
            
        #gonna be used for ranking purposes
        self.extra_segs = sorted(list(extra_segs), key=lambda x: x[1][0])
        
    #this removes any of the regex formations above and recalculates the new positions
    def remove_and_reass(self, rem = []):
        for r in rem:
            if r in song.regex_commands.keys():
                self.raw_text = re.sub(song.regex_commands[r], '', self.raw_text)
        self.assign_extras()
    
    #this make as a list that holds the segments in order. Each segment has a location and content of the segment
    #this list of segments creates the song
    def create_song_as_seg(self):
        self.segments = []
        for index, extra in enumerate(self.extra_segs):
            ex_type = extra[0][0]
            ex_cont = extra[0][1]
            ex_start = extra[1][0]
            ex_end = extra[1][1]
            
            if not self.segments:
                #this means on first segment
                seg = text_segment('[pre_text]', 'pre_text', 0, 0)
                #pull until the first first real segment
                seg.gen_content(self.raw_text, ex_start)
                self.segments = [seg]
                
            seg = text_segment(ex_type, ex_cont, ex_start, ex_end)
            #if we're on the last tag the end is the end of string
            if index+1 == len(self.extra_segs):
                #this means we're on the last segment
                seg.gen_content(self.raw_text, len(self.raw_text))
            else:
                #pull the next segments start position so we know where to end
                seg.gen_content(self.raw_text, self.extra_segs[index+1][1][0])
            self.segments.append(seg)


What you did last time:

made 99% method of capturing all verses - huge
tested regex and everything looks good (excluded verses from other stuff)
-made a couple changes here to capture more for intro, verse, chorus, outro, etc.
built all_aritist_names table (MUST DO ON OTHER PCS)
tested my segments/song creation, everything matches perfectly

next:
now can fuck around with chorus or proceed to cleaning verses/choruses
^if you do this then finish that, clean comments/code and make it into a library and move forward
scrape a few more artists

goals:
KEEP CLEANING AND LEANING

very close

In [12]:
song_list = []
for art in SCRAPED_RAPPERS:
    art_work = basic_lyrc_pull(estconn, art)
    for art, text in art_work.items():
        cur_song = song(text, art)
        cur_song.assign_extras()
        cur_song.remove_and_reass(['*text', '?'])
        cur_song.create_song_as_seg()
        song_list.append(cur_song)

In [19]:
estconn.close()